<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/clusterboot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade kmodes

Requirement already up-to-date: kmodes in /usr/local/lib/python3.7/dist-packages (0.11.0)


In [2]:
from google.colab import drive
import pathlib
import pandas as pd
import numpy as np
from pylab import *

from sklearn.preprocessing import StandardScaler

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

In [3]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [4]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
# Dataset de inventarios sin dimensiones de los productos
data1 = pd.read_csv("/content/gdrive/MyDrive/TDGdata/df042421.csv")

In [6]:
data1['AISLE']=data1['AISLE'].astype(str)

In [7]:
data1.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [8]:
data1.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029671,-0.037167
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092655,-0.006905
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266138,0.020477
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.024812,-0.044242
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077628,-0.046455


In [9]:
# scikit-learn bootstrap
from sklearn.utils import resample

In [10]:
data1.shape[0]

16244

In [11]:
catColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('object').columns)]

In [12]:
catColumnsPos

[0, 1, 2, 3, 4, 5, 6]

In [13]:
# Fit the cluster 
k = 4
gamma = 0.5
import time
start_time = time.time()
kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
kprototype.fit_predict(data1, categorical = catColumnsPos)
print("--- %s seconds ---" % (time.time() - start_time))

--- 73.27281022071838 seconds ---


In [14]:
data1['Cluster'] = kprototype.labels_

In [15]:
def add2set (nuevos):
  conjunto = {999999}
  conjunto.update(nuevos)
  conjunto.discard(999999)
  return(conjunto)


In [16]:
orig0 = {}
orig1 = {}
orig2 = {}
orig3 = {}

In [17]:
orig0 = add2set(data1[data1['Cluster']==0].index)
orig1 = add2set(data1[data1['Cluster']==1].index)
orig2 = add2set(data1[data1['Cluster']==2].index)
orig3 = add2set(data1[data1['Cluster']==3].index)

In [18]:
# Inicializamos la matriz de distancias con ceros
import time
start_time = time.time()
nBootstrap = 100
k = 4
gamma = 0.5
jaccard = [ [ 0 for i in range(k) ] for j in range(nBootstrap) ]
boot0 = {}
boot1 = {}
boot2 = {}
boot3 = {}

In [19]:
for i in range(nBootstrap):
  print('bootstrap # ',i);
  boot = resample(data1, replace=True, n_samples=data1.shape[0], random_state=i)
  # Fit the cluster
  kprototypebs = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 1)
  kprototypebs.fit_predict(boot, categorical = catColumnsPos)
  boot['Cluster'] = kprototypebs.labels_
  print("--- %s seconds ---" % (time.time() - start_time));
  boot0 = add2set(boot[boot['Cluster']==0].index)
  boot1 = add2set(boot[boot['Cluster']==1].index)
  boot2 = add2set(boot[boot['Cluster']==2].index)
  boot3 = add2set(boot[boot['Cluster']==3].index)
  jaccard[i][0]=max(len(orig0 & boot0) / len(orig0 | boot0), len(orig0 & boot1) / len(orig0 | boot1), len(orig0 & boot2) / len(orig0 | boot2), len(orig0 & boot3) / len(orig0 | boot3))
  jaccard[i][1]=max(len(orig1 & boot0) / len(orig1 | boot0), len(orig1 & boot1) / len(orig1 | boot1), len(orig1 & boot2) / len(orig1 | boot2), len(orig1 & boot3) / len(orig1 | boot3))
  jaccard[i][2]=max(len(orig2 & boot0) / len(orig2 | boot0), len(orig2 & boot1) / len(orig2 | boot1), len(orig2 & boot2) / len(orig2 | boot2), len(orig2 & boot3) / len(orig2 | boot3))
  jaccard[i][3]=max(len(orig3 & boot0) / len(orig3 | boot0), len(orig3 & boot1) / len(orig3 | boot1), len(orig3 & boot2) / len(orig3 | boot2), len(orig3 & boot3) / len(orig3 | boot3))
  print(jaccard[i][0:k]);

bootstrap #  0
--- 61.46764540672302 seconds ---
[0.33705342402430166, 0.6314699792960663, 0.010582010582010581, 0.6026058631921825]
bootstrap #  1
--- 117.43523168563843 seconds ---
[0.33350923482849604, 0.6066252587991718, 0.007751937984496124, 0.6182707993474714]
bootstrap #  2
--- 170.59232139587402 seconds ---
[0.6340883576570032, 0.6687370600414079, 0.5555555555555556, 0.6163934426229508]
bootstrap #  3
--- 234.70812344551086 seconds ---
[0.6364656937198706, 0.6211180124223602, 0.75, 0.5918032786885246]
bootstrap #  4
--- 292.1330509185791 seconds ---
[0.6305223535627023, 0.6169772256728778, 0.375, 0.6280587275693311]
bootstrap #  5
--- 347.3329186439514 seconds ---
[0.6321732813841379, 0.6438923395445134, 0.6666666666666666, 0.6491803278688525]
bootstrap #  6
--- 410.1875488758087 seconds ---
[0.6307204649012745, 0.6418219461697723, 0.5555555555555556, 0.6131147540983607]
bootstrap #  7
--- 460.56280851364136 seconds ---
[0.635607211252724, 0.6356107660455487, 0.8888888888888888

In [24]:
# np.savetxt("jaccard.csv", jaccard, delimiter=",")

In [27]:
# np.savetxt("/content/gdrive/MyDrive/TDGdata/jaccard.csv", jaccard, delimiter=",")

In [28]:
jaccard = pd.DataFrame(jaccard)

In [29]:
jaccard.describe()

,0,1,2,3
count,100.000000,100.000000,100.000000,100.000000
mean,0.580937,0.634099,0.452120,0.632613
std,0.112420,0.019382,0.261164,0.019266
min,0.328601,0.583851,0.004819,0.585366
25%,0.628376,0.619048,0.350694,0.619028
50%,0.630555,0.634576,0.500000,0.636066
75%,0.632702,0.648033,0.625000,0.647829
max,0.639503,0.681159,1.000000,0.667213
